This workbook parses the directions returned from google maps into the complete list of directions in tableau format.

In [40]:
import pandas as pd
import ast
import numpy as np
import random

In [41]:
# Reading in directions
directions = pd.read_csv('zone5_7trucks_directions_results.csv',index_col=0)

In [42]:
# Reading in queries
queries = pd.read_csv('zone5_7trucks_directions.csv',index_col=0)

In [47]:
# Merging the directions and queries to get the vehicleID for each direction
df = pd.merge(queries, directions, how='inner',left_on=['startHouses','endHouses'],right_on=['start','end'])
df = df[['start','end','vehicleID','directions']]
pairID = 0
ls = []
for index, row in df.iterrows():
    vehicleID = row['vehicleID']
    for command in ast.literal_eval(row['directions']):
        start = [command['start_location']['lng'], 
                 command['start_location']['lat'],
                 0,
                 pairID, 
                 vehicleID,
                 row['end'],'d']
        end = [command['end_location']['lng'], 
               command['end_location']['lat'],
               command['duration']['value'],
               pairID, 
               vehicleID,
               row['end'],'d']
        
        pairID += 1
        ls.extend([start,end])
        
    ls[-1][2] += 15

# Putting the results into a DF
directionRoutes = pd.DataFrame(ls, columns=['longitude','latitude','idvTime','pairID','vehicleID','location','type'])      

In [27]:
# Computing the running time for each truck
directionRoutes['time'] = directionRoutes.groupby('vehicleID')['idvTime'].cumsum()

In [28]:
# Manipulation the locations of the individual houses (so we can plot the locations in tableau)
singularLocations = queries[['endHouses','vehicleID','endLongitude','endLatitude']]
singularLocations['type'] = 'l'
singularLocations = singularLocations.rename(columns={'endLongitude':'longitude',
                                                      'endLatitude':'latitude',
                                                     'endHouses':'location'})
singularLocations['pairID'] = range(2000+0,2000+len(singularLocations))

<ipython-input-28-f871bc1cd21e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  singularLocations['type'] = 'l'


In [29]:
# Combining the datasets
directionRoutesComplete = pd.concat([directionRoutes,singularLocations])

In [30]:
directionRoutesComplete.to_csv('zone5_7trucks_tableau.csv')

In [31]:
undirectedRoute = pd.read_csv('zone5_7trucks.csv',index_col=0)

In [32]:
# Undirected Results
routes = [undirectedRoute[undirectedRoute['vehicleID']==i] for i in range(0,7)]
times = []
for route in routes:
    times.append(route.iloc[len(route)-1,]['time'] + len(route)*15)
print(times)
print(np.sum(times)/3600)

[1916, 2057, 12195, 11532, 11520, 12362, 7657]
16.455277777777777


In [34]:
# Directed Results
routes = [directionRoutes[directionRoutes['vehicleID']==i] for i in range(0,7)]
times = []
for route in routes:
    times.append(route.iloc[len(route)-1,]['time'])
print([i/3600 for i in times])
print(np.sum(times)/3600)

[0.2461111111111111, 0.2891666666666667, 2.0297222222222224, 1.988888888888889, 2.0772222222222223, 2.0275, 1.4172222222222222]
10.075833333333334


In [37]:
routes[1]

,longitude,latitude,idvTime,pairID,vehicleID,location,type,time
24,-122.762479,49.247505,0,12,1,"2230 KINGSWAY AVE, Port Coquitlam, BC, Canada",d,0
25,-122.762386,49.245638,14,12,1,"2230 KINGSWAY AVE, Port Coquitlam, BC, Canada",d,14
26,-122.762386,49.245638,0,13,1,"2230 KINGSWAY AVE, Port Coquitlam, BC, Canada",d,14
27,-122.756115,49.245637,59,13,1,"2230 KINGSWAY AVE, Port Coquitlam, BC, Canada",d,73
28,-122.756115,49.245637,0,14,1,"2230 KINGSWAY AVE, Port Coquitlam, BC, Canada",d,73
29,-122.756091,49.249799,54,14,1,"2230 KINGSWAY AVE, Port Coquitlam, BC, Canada",d,127
30,-122.756091,49.249799,0,15,1,"2230 KINGSWAY AVE, Port Coquitlam, BC, Canada",d,127
31,-122.777173,49.263311,240,15,1,"2230 KINGSWAY AVE, Port Coquitlam, BC, Canada",d,367
32,-122.777173,49.263311,0,16,1,"2236 KINGSWAY AVE, Port Coquitlam, BC, Canada",d,367
33,-122.777959,49.263595,21,16,1,"2236 KINGSWAY AVE, Port Coquitlam, BC, Canada",d,388
